In [1]:
import requests
import pandas as pd
import os
from pyspark.sql import SparkSession
from concurrent.futures import ThreadPoolExecutor, as_completed
import json
from dotenv import load_dotenv

In [2]:
load_dotenv()
TOKEN= os.getenv('TOKEN')


In [3]:
spark = SparkSession.builder.appName("API SP").getOrCreate()

25/05/12 19:10:31 WARN Utils: Your hostname, mecoaliza-Nitro-AN515-57 resolves to a loopback address: 127.0.1.1; using 192.168.0.4 instead (on interface wlp0s20f3)
25/05/12 19:10:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/12 19:10:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
key = TOKEN
pages = 5000

url = f'https://specieslink.net/ws/1.0/search?stateprovince=Alagoas&apikey={key}'


In [5]:
r = requests.get(f'{url}&offset=0&limit={pages}&apikey={key}')
r
#r.json()['features']

<Response [200]>

In [6]:
total_registros = r.json()['numberMatched']
total_registros

92128

In [7]:
total_json = []
page_init = 0

In [8]:
qtd_paginas = (total_registros // pages) + 1
qtd_paginas

19

In [9]:
sc = spark.sparkContext  # Obter o contexto Spark

In [10]:
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(requests.get, f'{url}&offset={page_init}&limit={pages}&apikey={key}') for page in range(1, qtd_paginas + 1)]

    for future in as_completed(futures):
        response = future.result()
        total_json.extend(response.json()['features'])

print(f"Total de registros obtidos: {len(total_json)}")

Total de registros obtidos: 95000


In [13]:
with open('../data_raw/all_records.json', 'w') as file:
    json.dump(total_json, file)